In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
nome_ativo = 'PETROBRAS'
ativo = "PETR3.SA"
data_final = datetime.today() - timedelta(days=(30))

In [ ]:
data_inicial = data_final - timedelta(days=(365+30))
dados1 = yf.download(ativo, start=data_inicial, end=data_final)

In [ ]:
data_inicial = data_final - timedelta(days=((365*5)+30))
dados5 = yf.download(ativo, start=data_inicial, end=data_final)

In [ ]:
data_inicial = data_final - timedelta(days=((365*10)+30))
dados10 = yf.download(ativo, start=data_inicial, end=data_final)

In [ ]:
data_inicial = data_final - timedelta(days=((365*20)+30))
dados20 = yf.download(ativo, start=data_inicial, end=data_final)

In [ ]:
dados1.reset_index(inplace=True)
dados1['mm9'] = 0
for i in range(9, len(dados1)):
    dados1['mm9'].loc[i] = dados1['Adj Close'].loc[i-9:i-1].mean()
dados1['mm14'] = 0
for i in range(14, len(dados1)):
    dados1['mm14'].loc[i] = dados1['Adj Close'].loc[i-14:i-1].mean()
dados1['mm21'] = 0
for i in range(21, len(dados1)):
        dados1['mm21'].loc[i] = dados1['Adj Close'].loc[i-21:i-1].mean()
dados1['vwap9'] = 0
for i in range(9, len(dados1)):
    soma, peso = 0,0
    for j in range(9):
        soma += dados1['Adj Close'].loc[i-j-1]*dados1['Volume'].loc[i-j-1]
        peso += dados1['Volume'].loc[i-j-1]
    dados1['vwap9'].loc[i] = soma/peso
dados1['vwap14'] = 0
for i in range(14, len(dados1)):
    soma, peso = 0,0
    for j in range(14):
        soma += dados1['Adj Close'].loc[i-j-1]*dados1['Volume'].loc[i-j-1]
        peso += dados1['Volume'].loc[i-j-1]
    dados1['vwap14'].loc[i] = soma/peso
dados1['vwap21'] = 0
for i in range(21, len(dados1)):
    soma, peso = 0,0
    for j in range(21):
        soma += dados1['Adj Close'].loc[i-j-1]*dados1['Volume'].loc[i-j-1]
        peso += dados1['Volume'].loc[i-j-1]
    dados1['vwap21'].loc[i] = soma/peso
dados1['linhak'] = 0
for i in range(14, len(dados1)):
    c = dados1['Adj Close'].loc[i]
    Ln = dados1['Low'].loc[i-14:i-1].min()
    Hn = dados1['High'].loc[i-14:i-1].max()
    dados1['linhak'].loc[i] = (c-Ln)/(Hn-Ln)*100
dados1['linhad'] = 0
for i in range(17, len(dados1)):
    ld = dados1['linhak'].loc[i-3:i-1].mean()
    dados1['linhad'].loc[i] = ld
dados1['rsi9'] = 0
for i in range(9, len(dados1)):
    dados1['rsi9'].loc[i] = 100-100/(1+((dados1['High'].loc[i-9:i-1].mean())/(dados1['Low'].loc[i-9:i-1].mean())))
dados1['rsi14'] = 0
for i in range(14, len(dados1)):
    dados1['rsi14'].loc[i] = 100-100/(1+((dados1['High'].loc[i-14:i-1].mean())/(dados1['Low'].loc[i-14:i-1].mean())))
dados1['rsi21'] = 0
for i in range(21, len(dados1)):
    dados1['rsi21'].loc[i] = 100-100/(1+((dados1['High'].loc[i-21:i-1].mean())/(dados1['Low'].loc[i-21:i-1].mean())))
dados1['linhamacd'] = 0
for i in range(26, len(dados1)):
    mme12 = dados1['Adj Close'].loc[i-26:i-15].mean()
    mme26 = dados1['Adj Close'].loc[i-26:i-1].mean()
    linhamacd = mme12 - mme26
    dados1['linhamacd'].loc[i] = linhamacd
dados1['sinalmacd'] = 0
for i in range(9, len(dados1)):
    mme9 = dados1['linhamacd'].loc[i-9:i-1].mean()
    dados1['sinalmacd'].loc[i] = mme9
dados1['bbsuperior'] = 0
dados1['bbinferior'] = 0
dados1['bbmeio'] = 0
for i in range(20, len(dados1)):
    lc = dados1['Adj Close'].loc[i-20:i-1].mean()
    dp = np.std(dados1['Adj Close'].loc[i-20:i-1])
    bs = lc+2*dp
    bi = lc-2*dp
    dados1['bbmeio'].loc[i] = lc
    dados1['bbsuperior'].loc[i] = bs
    dados1['bbinferior'].loc[i] = bi
dados1['target'] = 0
for i in range(1,len(dados1)):
    if dados1['Adj Close'].loc[i]>=dados1['Adj Close'].loc[i-1]:
        dados1['target'].loc[i] = 1
    else:
        dados1['target'].loc[i] = 0
dados1 = dados1[29:]

In [ ]:
dados5.reset_index(inplace=True)
dados5['mm9'] = 0
for i in range(9, len(dados5)):
    dados5['mm9'].loc[i] = dados5['Adj Close'].loc[i-9:i-1].mean()
dados5['mm14'] = 0
for i in range(14, len(dados5)):
    dados5['mm14'].loc[i] = dados5['Adj Close'].loc[i-14:i-1].mean()
dados5['mm21'] = 0
for i in range(21, len(dados5)):
        dados5['mm21'].loc[i] = dados5['Adj Close'].loc[i-21:i-1].mean()
dados5['vwap9'] = 0
for i in range(9, len(dados5)):
    soma, peso = 0,0
    for j in range(9):
        soma += dados5['Adj Close'].loc[i-j-1]*dados5['Volume'].loc[i-j-1]
        peso += dados5['Volume'].loc[i-j-1]
    dados5['vwap9'].loc[i] = soma/peso
dados5['vwap14'] = 0
for i in range(14, len(dados5)):
    soma, peso = 0,0
    for j in range(14):
        soma += dados5['Adj Close'].loc[i-j-1]*dados5['Volume'].loc[i-j-1]
        peso += dados5['Volume'].loc[i-j-1]
    dados5['vwap14'].loc[i] = soma/peso
dados5['vwap21'] = 0
for i in range(21, len(dados5)):
    soma, peso = 0,0
    for j in range(21):
        soma += dados5['Adj Close'].loc[i-j-1]*dados5['Volume'].loc[i-j-1]
        peso += dados5['Volume'].loc[i-j-1]
    dados5['vwap21'].loc[i] = soma/peso
dados5['linhak'] = 0
for i in range(14, len(dados5)):
    c = dados5['Adj Close'].loc[i]
    Ln = dados5['Low'].loc[i-14:i-1].min()
    Hn = dados5['High'].loc[i-14:i-1].max()
    dados5['linhak'].loc[i] = (c-Ln)/(Hn-Ln)*100
dados5['linhad'] = 0
for i in range(17, len(dados5)):
    ld = dados5['linhak'].loc[i-3:i-1].mean()
    dados5['linhad'].loc[i] = ld
dados5['rsi9'] = 0
for i in range(9, len(dados5)):
    dados5['rsi9'].loc[i] = 100-100/(1+((dados5['High'].loc[i-9:i-1].mean())/(dados5['Low'].loc[i-9:i-1].mean())))
dados5['rsi14'] = 0
for i in range(14, len(dados5)):
    dados5['rsi14'].loc[i] = 100-100/(1+((dados5['High'].loc[i-14:i-1].mean())/(dados5['Low'].loc[i-14:i-1].mean())))
dados5['rsi21'] = 0
for i in range(21, len(dados5)):
    dados5['rsi21'].loc[i] = 100-100/(1+((dados5['High'].loc[i-21:i-1].mean())/(dados5['Low'].loc[i-21:i-1].mean())))
dados5['linhamacd'] = 0
for i in range(26, len(dados5)):
    mme12 = dados5['Adj Close'].loc[i-26:i-15].mean()
    mme26 = dados5['Adj Close'].loc[i-26:i-1].mean()
    linhamacd = mme12 - mme26
    dados5['linhamacd'].loc[i] = linhamacd
dados5['sinalmacd'] = 0
for i in range(9, len(dados5)):
    mme9 = dados5['linhamacd'].loc[i-9:i-1].mean()
    dados5['sinalmacd'].loc[i] = mme9
dados5['bbsuperior'] = 0
dados5['bbinferior'] = 0
dados5['bbmeio'] = 0
for i in range(20, len(dados5)):
    lc = dados5['Adj Close'].loc[i-20:i-1].mean()
    dp = np.std(dados5['Adj Close'].loc[i-20:i-1])
    bs = lc+2*dp
    bi = lc-2*dp
    dados5['bbmeio'].loc[i] = lc
    dados5['bbsuperior'].loc[i] = bs
    dados5['bbinferior'].loc[i] = bi
dados5['target'] = 0
for i in range(1,len(dados5)):
    if dados5['Adj Close'].loc[i]>=dados5['Adj Close'].loc[i-1]:
        dados5['target'].loc[i] = 1
    else:
        dados5['target'].loc[i] = 0
dados5 = dados5[29:]

In [ ]:
dados10.reset_index(inplace=True)
dados10['mm9'] = 0
for i in range(9, len(dados10)):
    dados10['mm9'].loc[i] = dados10['Adj Close'].loc[i-9:i-1].mean()
dados10['mm14'] = 0
for i in range(14, len(dados10)):
    dados10['mm14'].loc[i] = dados10['Adj Close'].loc[i-14:i-1].mean()
dados10['mm21'] = 0
for i in range(21, len(dados10)):
        dados10['mm21'].loc[i] = dados10['Adj Close'].loc[i-21:i-1].mean()
dados10['vwap9'] = 0
for i in range(9, len(dados10)):
    soma, peso = 0,0
    for j in range(9):
        soma += dados10['Adj Close'].loc[i-j-1]*dados10['Volume'].loc[i-j-1]
        peso += dados10['Volume'].loc[i-j-1]
    dados10['vwap9'].loc[i] = soma/peso
dados10['vwap14'] = 0
for i in range(14, len(dados10)):
    soma, peso = 0,0
    for j in range(14):
        soma += dados10['Adj Close'].loc[i-j-1]*dados10['Volume'].loc[i-j-1]
        peso += dados10['Volume'].loc[i-j-1]
    dados10['vwap14'].loc[i] = soma/peso
dados10['vwap21'] = 0
for i in range(21, len(dados10)):
    soma, peso = 0,0
    for j in range(21):
        soma += dados10['Adj Close'].loc[i-j-1]*dados10['Volume'].loc[i-j-1]
        peso += dados10['Volume'].loc[i-j-1]
    dados10['vwap21'].loc[i] = soma/peso
dados10['linhak'] = 0
for i in range(14, len(dados10)):
    c = dados10['Adj Close'].loc[i]
    Ln = dados10['Low'].loc[i-14:i-1].min()
    Hn = dados10['High'].loc[i-14:i-1].max()
    dados10['linhak'].loc[i] = (c-Ln)/(Hn-Ln)*100
dados10['linhad'] = 0
for i in range(17, len(dados10)):
    ld = dados10['linhak'].loc[i-3:i-1].mean()
    dados10['linhad'].loc[i] = ld
dados10['rsi9'] = 0
for i in range(9, len(dados10)):
    dados10['rsi9'].loc[i] = 100-100/(1+((dados10['High'].loc[i-9:i-1].mean())/(dados10['Low'].loc[i-9:i-1].mean())))
dados10['rsi14'] = 0
for i in range(14, len(dados10)):
    dados10['rsi14'].loc[i] = 100-100/(1+((dados10['High'].loc[i-14:i-1].mean())/(dados10['Low'].loc[i-14:i-1].mean())))
dados10['rsi21'] = 0
for i in range(21, len(dados10)):
    dados10['rsi21'].loc[i] = 100-100/(1+((dados10['High'].loc[i-21:i-1].mean())/(dados10['Low'].loc[i-21:i-1].mean())))
dados10['linhamacd'] = 0
for i in range(26, len(dados10)):
    mme12 = dados10['Adj Close'].loc[i-26:i-15].mean()
    mme26 = dados10['Adj Close'].loc[i-26:i-1].mean()
    linhamacd = mme12 - mme26
    dados10['linhamacd'].loc[i] = linhamacd
dados10['sinalmacd'] = 0
for i in range(9, len(dados10)):
    mme9 = dados10['linhamacd'].loc[i-9:i-1].mean()
    dados10['sinalmacd'].loc[i] = mme9
dados10['bbsuperior'] = 0
dados10['bbinferior'] = 0
dados10['bbmeio'] = 0
for i in range(20, len(dados10)):
    lc = dados10['Adj Close'].loc[i-20:i-1].mean()
    dp = np.std(dados10['Adj Close'].loc[i-20:i-1])
    bs = lc+2*dp
    bi = lc-2*dp
    dados10['bbmeio'].loc[i] = lc
    dados10['bbsuperior'].loc[i] = bs
    dados10['bbinferior'].loc[i] = bi
dados10['target'] = 0
for i in range(1,len(dados10)):
    if dados10['Adj Close'].loc[i]>=dados10['Adj Close'].loc[i-1]:
        dados10['target'].loc[i] = 1
    else:
        dados10['target'].loc[i] = 0
dados10 = dados10[29:]

In [ ]:
dados20.reset_index(inplace=True)
dados20['mm9'] = 0
for i in range(9, len(dados20)):
    dados20['mm9'].loc[i] = dados20['Adj Close'].loc[i-9:i-1].mean()
dados20['mm14'] = 0
for i in range(14, len(dados20)):
    dados20['mm14'].loc[i] = dados20['Adj Close'].loc[i-14:i-1].mean()
dados20['mm21'] = 0
for i in range(21, len(dados20)):
        dados20['mm21'].loc[i] = dados20['Adj Close'].loc[i-21:i-1].mean()
dados20['vwap9'] = 0
for i in range(9, len(dados20)):
    soma, peso = 0,0
    for j in range(9):
        soma += dados20['Adj Close'].loc[i-j-1]*dados20['Volume'].loc[i-j-1]
        peso += dados20['Volume'].loc[i-j-1]
    dados20['vwap9'].loc[i] = soma/peso
dados20['vwap14'] = 0
for i in range(14, len(dados20)):
    soma, peso = 0,0
    for j in range(14):
        soma += dados20['Adj Close'].loc[i-j-1]*dados20['Volume'].loc[i-j-1]
        peso += dados20['Volume'].loc[i-j-1]
    dados20['vwap14'].loc[i] = soma/peso
dados20['vwap21'] = 0
for i in range(21, len(dados20)):
    soma, peso = 0,0
    for j in range(21):
        soma += dados20['Adj Close'].loc[i-j-1]*dados20['Volume'].loc[i-j-1]
        peso += dados20['Volume'].loc[i-j-1]
    dados20['vwap21'].loc[i] = soma/peso
dados20['linhak'] = 0
for i in range(14, len(dados20)):
    c = dados20['Adj Close'].loc[i]
    Ln = dados20['Low'].loc[i-14:i-1].min()
    Hn = dados20['High'].loc[i-14:i-1].max()
    dados20['linhak'].loc[i] = (c-Ln)/(Hn-Ln)*100
dados20['linhad'] = 0
for i in range(17, len(dados20)):
    ld = dados20['linhak'].loc[i-3:i-1].mean()
    dados20['linhad'].loc[i] = ld
dados20['rsi9'] = 0
for i in range(9, len(dados20)):
    dados20['rsi9'].loc[i] = 100-100/(1+((dados20['High'].loc[i-9:i-1].mean())/(dados20['Low'].loc[i-9:i-1].mean())))
dados20['rsi14'] = 0
for i in range(14, len(dados20)):
    dados20['rsi14'].loc[i] = 100-100/(1+((dados20['High'].loc[i-14:i-1].mean())/(dados20['Low'].loc[i-14:i-1].mean())))
dados20['rsi21'] = 0
for i in range(21, len(dados20)):
    dados20['rsi21'].loc[i] = 100-100/(1+((dados20['High'].loc[i-21:i-1].mean())/(dados20['Low'].loc[i-21:i-1].mean())))
dados20['linhamacd'] = 0
for i in range(26, len(dados20)):
    mme12 = dados20['Adj Close'].loc[i-26:i-15].mean()
    mme26 = dados20['Adj Close'].loc[i-26:i-1].mean()
    linhamacd = mme12 - mme26
    dados20['linhamacd'].loc[i] = linhamacd
dados20['sinalmacd'] = 0
for i in range(9, len(dados20)):
    mme9 = dados20['linhamacd'].loc[i-9:i-1].mean()
    dados20['sinalmacd'].loc[i] = mme9
dados20['bbsuperior'] = 0
dados20['bbinferior'] = 0
dados20['bbmeio'] = 0
for i in range(20, len(dados20)):
    lc = dados20['Adj Close'].loc[i-20:i-1].mean()
    dp = np.std(dados20['Adj Close'].loc[i-20:i-1])
    bs = lc+2*dp
    bi = lc-2*dp
    dados20['bbmeio'].loc[i] = lc
    dados20['bbsuperior'].loc[i] = bs
    dados20['bbinferior'].loc[i] = bi
dados20['target'] = 0
for i in range(1,len(dados20)):
    if dados20['Adj Close'].loc[i]>=dados20['Adj Close'].loc[i-1]:
        dados20['target'].loc[i] = 1
    else:
        dados20['target'].loc[i] = 0
dados20 = dados20[29:]

In [ ]:
metricas = [['mm9','mm14','mm21'], 
            ['vwap9','vwap14','vwap21'], 
            ['linhak','linhad'], 
            ['rsi9','rsi14','rsi21'], 
            ['linhamacd','sinalmacd'], 
            ['bbsuperior','bbinferior','bbmeio'], 
            ['Open']]
nomes = ['MEDIASMOVEIS', 
         'MEDIAPONDERADAPORVOLUME', 
         'CALCULOESTOCASTICO', 
         'RSI', 'MACD', 
         'BANDASBOLLINGER', 
         'OPEN']

In [ ]:
models = [LogisticRegression(),
          RandomForestClassifier(),
          SVC(),
          KNeighborsClassifier(),
          MLPClassifier()]

for m in models:
    for metrica in range(len(metricas)):
        model = m
        X = metricas[metrica]
        Y = ['target']
        X = dados1[X]
        Y = dados1[Y]
        trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=42)
        model.fit(trainX,trainY)
        previsao = model.predict(testX)
        print(accuracy_score(testY, previsao))

In [ ]:
models = [LogisticRegression(),
          RandomForestClassifier(),
          SVC(),
          KNeighborsClassifier(),
          MLPClassifier()]

for m in models:
    for metrica in range(len(metricas)):
        model = m
        X = metricas[metrica]
        Y = ['target']
        X = dados5[X]
        Y = dados5[Y]
        trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=42)
        model.fit(trainX,trainY)
        previsao = model.predict(testX)
        print(accuracy_score(testY, previsao))

In [ ]:
models = [LogisticRegression(),
          RandomForestClassifier(),
          SVC(),
          KNeighborsClassifier(),
          MLPClassifier()]

for m in models:
    for metrica in range(len(metricas)):
        model = m
        X = metricas[metrica]
        Y = ['target']
        X = dados10[X]
        Y = dados10[Y]
        trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=42)
        model.fit(trainX,trainY)
        previsao = model.predict(testX)
        print(accuracy_score(testY, previsao))

In [ ]:
models = [LogisticRegression(),
          RandomForestClassifier(),
          SVC(),
          KNeighborsClassifier(),
          MLPClassifier()]

for m in models:
    for metrica in range(len(metricas)):
        model = m
        X = metricas[metrica]
        Y = ['target']
        X = dados20[X]
        Y = dados20[Y]
        trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=42)
        model.fit(trainX,trainY)
        previsao = model.predict(testX)
        print(accuracy_score(testY, previsao))